In [1]:
from speech_encoder_v2 import SpeechEncoderV2
from params import *
from pathlib import Path
import torch
import utils
import visualisations
from data_processor import *

### Setting Up Profiler and Selecting the appropriate Device

In [2]:
def sync(device: torch.device):
    # For correct profiling (cuda operations are async)
    if device.type == "cuda":
        torch.cuda.synchronize(device)

### Setting up the Parameters

In [3]:
params = {
    "run_id": "speech_encoder_transformer",  # A unique identifier for this training run
    "clean_data_root": "D:/CODING/SpeechEncoder/data/processed_audio",  # Path to LibriSpeech dataset
    "models_dir": "models",  # Directory to save model checkpoints
    "umap_every": 500,  # Update UMAP visualization every 500 steps
    "save_every": 500,  # Save model checkpoint every 1000 steps
    "backup_every": 5000,  # Create a backup copy of the model every 5000 steps
    "vis_every": 100,  # Update visualization metrics every 100 steps
    "force_restart": False,  # Whether to restart training from scratch
    "visdom_server": "http://localhost",  # Visdom server address for visualization
    "no_visdom": False,  # Whether to disable Visdom visualization
    "models_dir": Path("models"),  # Directory to save model checkpoints
}

### Loading the data

In [4]:
dataset = SpeakerVerificationDataset(Path("D:/CODING/SpeechEncoder/data/his_processed_audio"))
loader = SpeakerVerificationDataLoader(
        dataset,
        40,
        10,
        num_workers=0,
    )

In [6]:
for batch in loader:
    print(batch.data.shape)
    break  # Check the shape of the first batch only

(400, 160, 40)


### Initializing the model

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_device = torch.device("cpu")

# Create the model and the optimizer
model = SpeechEncoderV2(device, device)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate_init)
init_step = 1

# Configure file path for the model
model_dir = params['models_dir'] / params['run_id']
model_dir.mkdir(exist_ok=True, parents=True)
state_fpath = model_dir / "encoder.pt"

c:\Users\anike\anaconda3\envs\speech_encoder\lib\site-packages\torch\nn\modules\transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


### Training the model

In [ ]:
import sys
from tqdm import tqdm  # Import tqdm for the progress bar

# Initialize the progress bar
total_steps = len(loader)  # Assuming `loader` has a defined length
progress_bar = tqdm(enumerate(loader, init_step), total=total_steps, desc="Training", unit="step")

for step, speaker_batch in progress_bar:
    # Forward pass
    inputs = torch.from_numpy(speaker_batch.data).to(device)
    sync(device)
    embeds = model(inputs)
    sync(device)
    embeds_loss = embeds.view((speakers_per_batch, utterances_per_speaker, -1)).to(loss_device)
    loss, eer = model.loss(embeds_loss)
    sync(loss_device)

    # Backward pass
    model.zero_grad()
    loss.backward()
    
    model.do_gradient_ops()
    optimizer.step()
    
    # Update the progress bar with the current loss
    progress_bar.set_postfix({"loss": loss.item(), "eer": eer})

    # Overwrite the latest version of the model
    if params['save_every'] != 0 and step % params['save_every'] == 0:
        print("Saving the model (step %d)" % step)
        torch.save({
            "step": step + 1,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
        }, state_fpath)

    # Make a backup
    if params['backup_every'] != 0 and step % params['backup_every'] == 0:
        print("Making a backup (step %d)" % step)
        backup_fpath = model_dir / f"encoder_{step:06d}.bak"
        torch.save({
            "step": step + 1,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
        }, backup_fpath)
    
    # Update the progress bar with loss and EER information.
    progress_bar.set_postfix(loss=loss.item(), eer=eer)


Training:   0%|          | 500/250000000 [05:43<38995:14:32,  1.78step/s, eer=0.114, loss=1.59] 

Saving the model (step 500)


Training:   0%|          | 1000/250000000 [10:20<38546:25:14,  1.80step/s, eer=0.095, loss=1.27]

Saving the model (step 1000)


Training:   0%|          | 1500/250000000 [14:55<38611:15:07,  1.80step/s, eer=0.0844, loss=1.15] 

...

Training:   0%|          | 57000/250000000 [9:54:38<38126:18:37,  1.82step/s, eer=0.015, loss=0.152]   

Saving the model (step 57000)


Training:   0%|          | 57500/250000000 [9:59:12<38336:11:04,  1.81step/s, eer=0.0171, loss=0.151]  

Saving the model (step 57500)


Training:   0%|          | 57948/250000000 [10:03:17<46804:47:13,  1.48step/s, eer=0.00904, loss=0.0793]